# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

# First Part

In [3]:
import numpy as np
import pandas as pd
import requests

## Separating the DataSet labels in columns :

In [5]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=947772202'
dataset = pd.read_html(url,  attrs={'rules': 'all'})
df = dataset[0]
list_names_of_borough = ['Scarborough','North York','East York','East Toronto','Central Toronto','Downtown Toronto','York','West Toronto','Etobicoke',"Queen's Park"]


for a,b in enumerate(df):
    for i,j in enumerate(df[a]):
        latter = 0
        storage = df[a][i]
        df[a][i] = ''
        #print(i,"Round")
        if 'Not assigned' in storage:
            df.drop(index = [a,i])
            continue # skips a iteration
        df[a][i] = storage[0:3] + ',' 
        if list_names_of_borough[a] in storage:
            for let in list_names_of_borough[a]:
                latter += 1
            first = storage.find(list_names_of_borough[a])
            
        else:
            for element in list_names_of_borough:
                if element in storage:
                    for let in element:
                        latter += 1
                    first = storage.find(element)
                    break
        latter += 3
        df[a][i] = df[a][i] + storage[first:latter] + ','
        df[a][i] = df[a][i] + storage[latter + 1:]   

## Cleaning the DataSet

In [6]:
table = str.maketrans(dict.fromkeys("()"))    
dic = {}
for a,b in enumerate(df):
    for i,j in enumerate(df[a]):
        df[a][i] = df[a][i].translate(table)
        df[a][i] = df[a][i].split(',')        
        if len(df[a][i]) != 0:
            dic[df[a][i][0]] = df[a][i][1:]

            
dic.pop('', [])
dic['M5B'] = ['Downtown Toronto' ,'Garden District / Ryerson']
list_of_index = []
list_of_borough = []
list_of_neigh = []
for key in dic:
    list_of_index.append(key)
    list_of_borough.append(dic[key][0])
    list_of_neigh.append(dic[key][1])
    

## Creating a DataFrame

In [7]:
real_df = pd.DataFrame(data = {'Postal Code' : list_of_index,'Borough':list_of_borough,'Neighborhood':list_of_neigh})
real_df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
real_df.tail()

,Postal Code,Borough,Neighborhood
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...
102,M9W,Etobicoke,orthwestClairville / Humberwood / Woodbine Dow...


In [9]:
real_df.shape

(103, 3)

# Second Part

In [10]:
url = 'https://cocl.us/Geospatial_data'
df = pd.read_csv(url)
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
# Checking if both columns are equal
for index,postal in enumerate(df.iloc[0:,0]):
    if real_df.iloc[index,0] != postal:
        print(real_df.iloc[index,0],' != ',postal)
# joining the datasets and merging the postal code column
real_df = real_df.join(df.iloc[0:,1:])


In [12]:
real_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [13]:
real_df.tail()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...,43.688905,-79.554724
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...,43.739416,-79.588437
102,M9W,Etobicoke,orthwestClairville / Humberwood / Woodbine Dow...,43.706748,-79.594054


# Third Part

In [33]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters= 4, random_state=0).fit(real_df.drop(['Neighborhood','Borough','Postal Code'], 1))
kmeans.labels_

array([3, 3, 3, 3, 3, 3, 3, 1, 3, 1, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [34]:
cluster_df = real_df
cluster_df['Clusters'] = kmeans.labels_
cluster_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Clusters
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,3
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,3
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,3
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3


## Clusters

In [48]:
# First Cluster
cluster_df.loc[cluster_df['Clusters'] == 0, cluster_df.columns[[0,1,2] + list(range(4, cluster_df.shape[1]))]]

,Postal Code,Borough,Neighborhood,Longitude,Clusters
17,M2H,North York,Hillcrest Village,-79.363452,0
18,M2J,North York,Fairview / Henry Farm / Oriole,-79.346556,0
19,M2K,North York,Bayview Village,-79.385975,0
20,M2L,North York,York Mills / Silver Hills,-79.374714,0
21,M2M,North York,Willowdale / Newtonbrook,-79.408493,0
22,M2N,North York,WillowdaleSouth,-79.408493,0
23,M2P,North York,York Mills West,-79.400049,0
24,M2R,North York,WillowdaleWest,-79.442259,0
28,M3H,North York,Bathurst Manor / Wilson Heights / Downsview North,-79.442259,0
29,M3J,North York,Northwood Park / York University,-79.487262,0


In [49]:
# Second Cluster
cluster_df.loc[cluster_df['Clusters'] == 1, cluster_df.columns[[0,1,2] + list(range(4, cluster_df.shape[1]))]]

,Postal Code,Borough,Neighborhood,Longitude,Clusters
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,-79.284577,1
9,M1N,Scarborough,Birch Cliff / Cliffside West,-79.264848,1
25,M3A,North York,Parkwoods,-79.329656,1
26,M3B,North York,Don MillsNorth,-79.352188,1
27,M3C,North York,Don MillsSouthFlemingdon Park,-79.340923,1
34,M4A,North York,Victoria Village,-79.315572,1
35,M4B,East York,Parkview Hill / Woodbine Gardens,-79.309937,1
36,M4C,East York,Woodbine Heights,-79.318389,1
37,M4E,East Toronto,The Beaches,-79.293031,1
38,M4G,East York,Leaside,-79.363452,1


In [50]:
# Third Cluster
cluster_df.loc[cluster_df['Clusters'] == 2, cluster_df.columns[[0,1,2] + list(range(4, cluster_df.shape[1]))]]

,Postal Code,Borough,Neighborhood,Longitude,Clusters
31,M3L,North York,DownsviewWest,-79.506944,2
32,M3M,North York,DownsviewCentral,-79.495697,2
33,M3N,North York,DownsviewNorthwest,-79.520999,2
79,M6L,North York,North Park / Maple Leaf Park / Upwood Park,-79.490074,2
80,M6M,York,Del Ray / Mount Dennis / Keelsdale and Silvert...,-79.476013,2
81,M6N,York,Runnymede / The Junction North,-79.487262,2
84,M6S,West Toronto,Runnymede / Swansea,-79.484450,2
86,M7R,,ississaugaCanada Post Gateway Processing Centr...,-79.615819,2
88,M8V,Etobicoke,New Toronto / Mimico South / Humber Bay Shores,-79.501321,2
89,M8W,Etobicoke,Alderwood / Long Branch,-79.543484,2


In [51]:
# Fourth Cluster
cluster_df.loc[cluster_df['Clusters'] == 3, cluster_df.columns[[0,1,2] + list(range(4, cluster_df.shape[1]))]]

,Postal Code,Borough,Neighborhood,Longitude,Clusters
0,M1B,Scarborough,Malvern / Rouge,-79.194353,3
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,-79.160497,3
2,M1E,Scarborough,Guildwood / Morningside / West Hill,-79.188711,3
3,M1G,Scarborough,Woburn,-79.216917,3
4,M1H,Scarborough,Cedarbrae,-79.239476,3
5,M1J,Scarborough,Scarborough Village,-79.239476,3
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,-79.262029,3
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,-79.239476,3
10,M1P,Scarborough,Dorset Park / Wexford Heights / Scarborough To...,-79.273304,3
11,M1R,Scarborough,Wexford / Maryvale,-79.295849,3
